# Mongolia Lineaments #

Using [geemap](https://geemap.org/) to analyze lineaments generated from Landsat 7 and SRTM in Mongolia (and beyond). Edges (lineaments) were detected using [Canny Edge Detector](https://developers.google.com/earth-engine/apidocs/ee-algorithms-cannyedgedetector?hl=en) and further analyzed through [Hough Transform](https://developers.google.com/earth-engine/apidocs/ee-algorithms-houghtransform?hl=en).

Layer tab in top right can turn on/off layers as well as change opacity.

To start, just click submit in the bottom to get produce some lineaments.

Change the thresholds and sigma and click submit to change the visualization.

Data Sources -
* [SRTM](https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001?hl=en) (blue)
* [Landsat 7](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1?hl=en) (red)

In [1]:
# Import earth engine and geemap

import ee
import geemap
import ipywidgets as widgets
geemap.ee_initialize()

#############Import Shapefiles###########
mongoShape = ee.FeatureCollection("users/pirrie/MongoliaAssets/MongoliaOutline")
potSuture = ee.FeatureCollection("users/pirrie/MongoliaAssets/PotentialSuture")
mongoGeom = mongoShape.geometry()

Map = geemap.Map()
Map.centerObject(mongoShape, 5)

# Add Mongolia Shape, Suture
Map.addLayer(mongoShape, {'color': '000000'}, 'Mongolia', True, 0.3)
Map.addLayer(potSuture, {'color': 'FFFF00'}, 'Suture (Zorin)', True, 0.3)

# Load Landsat 7 data, filter by date and bounds.
collection = ee.ImageCollection("LANDSAT/LE07/C01/T1") \
  .filterDate('2019-01-01', '2021-08-18') \
  .filter(ee.Filter.calendarRange(6, 9, 'month'))

# Also filter the collection by the IMAGE_QUALITY property.
filtered = collection \
  .filterMetadata('IMAGE_QUALITY', 'equals', 9)

# Create two composites to check the effect of filtering by IMAGE_QUALITY.
goodComposite = ee.Algorithms \
  .Landsat.simpleComposite(filtered, 75, 3)

# Add PC band
Panchromatic = goodComposite \
  .select(['B8']) # \
  # .clip(mongoShape) \
  # .select(['B8'])

Map.addLayer(Panchromatic,
             {'gain': 1.5},
             'Panchromatic',
             False)

# Add SRTM 30m (https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001?hl=en)
DEM = ee.Image('NASA/NASADEM_HGT/001') \
  .select('elevation')

Map

Map(center=[46.966026359517194, 103.13830720118435], controls=(WidgetControl(options=['position', 'transparent…

>*Yellow line is proposed suture after Zorin (1999)*

### Threshold ###
The pixel is only considered for edge detection if the gradient magnitude is higher than this threshold.

In [2]:
style = {'description_width': 'initial'}

In [3]:
LSthresh = widgets.BoundedIntText(
  value=10,
  min=0,
  max=100,
  step=1,
  description='Landsat Threshold',
  disabled=False,
  style=style
)

DEMthresh = widgets.BoundedIntText(
  value=10,
  min=0,
  max=100,
  step=1,
  description='DEM Threshold',
  disabled=False,
  style=style
)

syncThresh = widgets.Checkbox(
  value=False,
  description='Sync Thresholds?',
  style=style
)

hbox1 = widgets.HBox([LSthresh, DEMthresh])
hbox1

### Sigma ###
Sigma value for a gaussian filter applied before edge detection. 0 means apply no filtering.

In [4]:
LSsigma = widgets.BoundedFloatText(
  value=1.5,
  min=0,
  max=10,
  step=0.1,
  description='Landsat Sigma',
  disabled=False,
  style=style
)

DEMsigma = widgets.BoundedFloatText(
  value=1.5,
  min=0,
  max=10,
  step=0.1,
  description='DEM Sigma',
  disabled=False,
  style=style
)

syncSigma = widgets.Checkbox(
  value=False,
  description='Sync Sigmas?',
  style=style
)

hbox2 = widgets.HBox([LSsigma, DEMsigma])
hbox2

In [5]:
submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click the submit the request to create lineaments',
    style=style
)

output = widgets.Output()

In [6]:
def edge_detect(LandsatThresh, SRTMThresh, LandsatSigma, SRTMSigma):

    #############Edge Detection###########

    # Detect edges in the panchromatic composite. Variables are (image, threshold, sigma)

    cannyLS = ee.Algorithms \
      .CannyEdgeDetector(Panchromatic, LandsatThresh, LandsatSigma)
    cannyDEM = ee.Algorithms \
      .CannyEdgeDetector(DEM, SRTMThresh, SRTMSigma)


    # Mask the image with itself to get rid of areas with no edges.
    cannyLS = cannyLS \
      .updateMask(cannyLS)
    cannyDEM = cannyDEM \
      .updateMask(cannyDEM)

    Map.addLayer(cannyLS, {'min': 0, 'max': 1, 'palette': '0620DB'}, 'CE Pan LS', False)
    Map.addLayer(cannyDEM, {'min': 0, 'max': 1, 'palette': '8F1713'}, 'CE DEM', False)

    #############Hough Transformation###########

    # Perform Hough transform of the Canny result and display.
    houghLS = ee.Algorithms \
      .HoughTransform(cannyLS, 256, 64, 300, True)
    houghDEM = ee.Algorithms \
      .HoughTransform(cannyDEM, 256, 64, 300, True)

    Map.addLayer(houghLS, {'palette': '1293DB'}, 'Hough LS', True, 0.5)
    Map.addLayer(houghDEM, {'palette': 'DB3832'}, 'Hough DEM', True, 0.5)

    bufferSize = 250
    houghLSbuffer = houghLS \
      .focal_max(bufferSize, 'square', 'meters')
    houghDEMbuffer = houghDEM \
      .focal_max(bufferSize, 'square', 'meters')

    Map.addLayer(houghLSbuffer, {'palette': '1293DB'}, 'Hough LS w/Buffer', False, 0.5)
    Map.addLayer(houghDEMbuffer, {'palette': 'DB3832'}, 'Hough DEM w/Buffer', False, 0.5)

In [7]:
def submit_clicked(b):

    with output:
        output.clear_output()
        print('Computing...')

        edge_detect(LandsatThresh=LSthresh.value, SRTMThresh=DEMthresh.value,
                    LandsatSigma=LSsigma.value, SRTMSigma=DEMsigma.value)

submit.on_click(submit_clicked)

In [8]:
submit

Button(button_style='primary', description='Submit', style=ButtonStyle(), tooltip='Click the submit the reques…

In [9]:
output

Output()